In [10]:
import kagglehub
# Download latest version
path = kagglehub.model_download("google/universal-sentence-encoder/tensorFlow2/universal-sentence-encoder")

print("Path to model files:", path)


  0%|                                                                    | 0.00/7.84M [00:00<?, ?B/s]



  0%|                                                                     | 0.00/980M [00:00<?, ?B/s]




100%|████████████████████████████████████████████████████████████| 3.11k/3.11k [00:00<00:00, 701kB/s]

 13%|███████▋                                                    | 1.00M/7.84M [00:01<00:12, 596kB/s]
 26%|███████████████                                            | 2.00M/7.84M [00:02<00:05, 1.04MB/s]
 38%|██████████████████████▌                                    | 3.00M/7.84M [00:02<00:03, 1.39MB/s]
 51%|██████████████████████████████                             | 4.00M/7.84M [00:03<00:02, 1.42MB/s]

  0%|                                                             | 1.00M/980M [00:02<44:29, 384kB/s]
 64%|█████████████████████████████████████▌                     | 5.00M/7.84M [00:04<00:02, 1.31MB/s]
 77%|█████████████████████████████████████████████▏             | 6.00M/7.84M [00:04<00:01, 1.38MB/s]

  0%|                                                             | 2.00M/980M [00:04<38:50, 440kB/s]
 89%|████████████████████████████████████████████████████▋      | 7.00M/7.84

Path to model files: C:\Users\sysadmin\.cache\kagglehub\models\google\universal-sentence-encoder\tensorFlow2\universal-sentence-encoder\2


In [11]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [12]:
# Version and model save directory
version = "0.1"
model_save_dir = f"./intent_recognition_model_v{version}/"

print("ai.m9j.github.io - Intent Recognition")

# Define intents
intents = {
    "greeting": ["hello", "hi", "hey"],
    "goodbye": ["bye", "goodbye", "see you later"],
    "thanks": ["thank you", "thanks", "much appreciated"],
}

ai.m9j.github.io - Intent Recognition


In [15]:
# Initialize the Universal Sentence Encoder
print("Loading USE...")
# embedder = hub.load("C:\Users\sysadmin\.cache\kagglehub\models\google\universal-sentence-encoder\tensorFlow2\universal-sentence-encoder\2")
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")
print("USE loaded.")

Loading USE...



USE loaded.


In [16]:
# Prepare training data
training_data = [
    {"text": "hello", "label": [1, 0, 0]},  # Greeting
    {"text": "hi", "label": [1, 0, 0]},     # Greeting
    {"text": "bye", "label": [0, 1, 0]},    # Goodbye
    {"text": "goodbye", "label": [0, 1, 0]},# Goodbye
    {"text": "thank you", "label": [0, 0, 1]}, # Thanks
    {"text": "thanks", "label": [0, 0, 1]},    # Thanks
]

# Generate embeddings for training data
print("Generating embeddings...")
training_texts = [data["text"] for data in training_data]
training_labels = np.array([data["label"] for data in training_data])
# training_embeddings = embedder(training_texts).numpy()
training_embeddings = embed(training_texts)
print("Embeddings generation completed.")

# Define the model
embedding_size = training_embeddings.shape[1]
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(embedding_size,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(intents), activation="softmax"),
])

Generating embeddings...
Embeddings generation completed.


In [18]:
# Compile the model
print("Compiling model...")
model.compile(optimizer="adam",
              # loss="categorical_crossentropy",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
print("Model compiled.")

Compiling model...
Model compiled.


In [19]:
# Train the model
print("Training model...")
history = model.fit(
    training_embeddings,
    training_labels,
    epochs=100,
    verbose=1  # Shows progress for each epoch
)
print("Model training completed.")

Training model...
Epoch 1/100


C:\python310\lib\site-packages\keras\src\backend\tensorflow\nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 3), output.shape=(None, 3)

In [ ]:
# Save the model
print(f"Saving the model to {model_save_dir}...")
model.save(model_save_dir)
print("Model saved.")

In [ ]:
# Predict an intent
input_texts = ["hello"]
input_embeddings = embedder(input_texts).numpy()
predictions = model.predict(input_embeddings)
predicted_index = np.argmax(predictions[0])
intent_labels = list(intents.keys())  # ['greeting', 'goodbye', 'thanks']
print(f"\nPredicted intent: {intent_labels[predicted_index]}")